In [7]:
import torch
import numpy as np
import random
import copy
import datetime
import FbsEnv
import gym
import FbsEnv.utils.FBSUtil as FBSUtil
from FbsEnv.utils.FBSUtil import FBSUtils
from loguru import logger
import FbsEnv.utils.ExperimentsUtil as ExperimentsUtil
import warnings
from stable_baselines3 import DQN
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

warnings.filterwarnings("ignore", module="gym")  # 忽略gym的警告

In [8]:
def train_id():
    time_str = datetime.datetime.now().strftime("%Y年%m月%d日%H时%M分")
    return time_str

In [9]:
class GeneticAlgorithm:
    def __init__(
        self,
        env,
        population_size=50,
        crossover_rate=0.8,
        mutation_rate=0.1,
        max_generations=100,
        dqn_train_freq=1,
        dqn_learning_starts=1000,
        device="auto",
        is_train=True,
        model_path=None
    ):
        self.env = env
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.max_generations = max_generations
        self.is_train = is_train
        if is_train:
            self.dqn_model = DQN(
                "MlpPolicy",  # 使用MLP策略
                env,
            policy_kwargs=dict(
                net_arch=[512, 256, 256, 128],  # 网络结构
                activation_fn=nn.ReLU
            ),
            verbose=0,
            learning_rate=5e-4,
            buffer_size=200000,  # 缓冲区大小
            learning_starts=dqn_learning_starts,
            batch_size=64,  # 批次大小
            train_freq=dqn_train_freq,
            gradient_steps=32,  # 梯度步数
            target_update_interval=1000,  # 目标网络更新频率
            exploration_fraction=0.1,  # 探索率衰减
            exploration_initial_eps=1.0,  # 初始探索率
            exploration_final_eps=0.05,  # 最终探索率
            device=device  # 指定训练设备
            )
        else:
            self.dqn_model = DQN.load(model_path, env=env)
        self.population = self.initialize_population()
        self.train_id = train_id()

    def initialize_population(self):
        """
        初始化种群，生成随机解
        :return: 种群列表，每个个体包含设施排列和条带划分
        """
        population = []
        for _ in range(self.population_size):
            individual = gym.make("FbsEnv-v0", instance=self.env.instance)
            individual.reset()
            population.append(individual)
        return population

    def evaluate_fitness(self, individual):
        """
        评估个体的适应度（MHC）
        :param individual: 个体
        :return: 适应度值（越小越好）
        """
        return individual.fitness

    def select(self):
        """
        选择操作：使用锦标赛选择法
        :return: 选择的个体
        """
        tournament_size = 5
        tournament = random.sample(self.population, tournament_size)
        best_in_tournament = min(tournament, key=lambda ind: self.evaluate_fitness(ind))
        return best_in_tournament

    def crossover(self, parent1, parent2):
        """
        交叉操作：对permutation使用OX交叉，对bay使用单点交叉
        :param parent1: 父代个体1
        :param parent2: 父代个体2
        :return: 子代个体1和子代个体2
        """
        if random.random() < self.crossover_rate:
            parent1_fbs_model = parent1.fbs_model
            parent2_fbs_model = parent2.fbs_model
            offspring1_fbs_model, offspring2_fbs_model = FBSUtils.CrossoverActions.order_crossover(
                parent1_fbs_model, parent2_fbs_model
            )
            offspring1 = gym.make("FbsEnv-v0", instance=self.env.instance)
            offspring2 = gym.make("FbsEnv-v0", instance=self.env.instance)
            offspring1.reset(fbs_model=offspring1_fbs_model)
            offspring2.reset(fbs_model=offspring2_fbs_model)
            return offspring1, offspring2
        else:
            return parent1, parent2
    def rl_mutate(self, individual):
        """
        强化学习引导的变异操作
        使用DQN模型选择最优动作来变异个体
        """
        if random.random() < self.mutation_rate:
            try:
                # 获取原始状态，确保形状正确
                original_state = individual.state
                # 使用DQN模型预测动作
                action, _ = self.dqn_model.predict(original_state, deterministic=False)
                # 执行变异操作
                new_state, reward, done, info = individual.step(action)
                if self.is_train:
                    # 直接训练DQN模型
                    if (self.dqn_model.num_timesteps > self.dqn_model.learning_starts and 
                        self.dqn_model.num_timesteps % 10 == 0):
                    # 使用随机样本训练，而不是依赖具体的回放缓冲区
                        self.dqn_model.learn(gradient_steps=32, batch_size=64)
            except Exception as e:
                logger.error(f"RL变异操作发生错误: {e}")
                # 失败时回退到普通变异
                individual = self.mutate(individual)
        return individual

    def mutate(self, individual):
        """
        变异操作：
        :param individual: 个体
        :return: 变异后的个体
        """
        if random.random() < self.mutation_rate:
            action  = np.random.randint(0, 3)  # 随机选择变异类型
            individual.step(action)  # 执行变异操作
            return individual
        else:
            return individual

    # TODO 修复局部优化问题
    def local_optimize(self, individual):
        """
        对个体进行局部优化
        :param individual: 需要优化的个体
        :return: 优化后的个体
        """
        env = gym.make("FbsEnv-v0", instance=self.env.instance)
        env.reset(fbs_model=individual.fbs_model)
        fac_list = FBSUtil.permutationToArray(env.fbs_model.permutation, env.fbs_model.bay)
        bay_index = np.random.choice(len(fac_list))
        logger.info("进行shuffle优化")
        permutation, bay = FBSUtil.shuffleOptimization(env, bay_index)
        optimized_fbs_model = FBSUtil.FBSModel(permutation, bay)
        optimized_individual = gym.make("FbsEnv-v0", instance=self.env.instance)
        optimized_individual.reset(fbs_model=optimized_fbs_model)
        return optimized_individual

    def run(self):
        """
        运行遗传算法，添加对优秀解的局部优化
        :return: 最优解、最优适应度、开始时间、结束时间、最优解发现时间
        """
        start_time = datetime.datetime.now()
        best_fitness = float("inf")
        best_solution = None
        best_dqn_model = None
        best_generation = 0
        best_time = None  # 添加最佳解的时间变量
        
        # 启用DQN的在线学习
        self.dqn_model.learn(total_timesteps=0)

        for generation in range(self.max_generations):
            fitness_values = [self.evaluate_fitness(ind) for ind in self.population]  # 计算种群适应度值
            elite_indices = np.argsort(fitness_values)[: int(self.population_size * 0.1)]  # 选择精英个体
            elite_population = [self.population[idx] for idx in elite_indices]
            new_population = elite_population[:]  # 保留精英个体

            while len(new_population) < self.population_size:
                parent1 = self.select()
                parent2 = self.select()
                child1, child2 = self.crossover(parent1, parent2)
                # child1 = self.mutate(child1)  # 使用普通变异替代RL变异
                # child2 = self.mutate(child2)  # 使用普通变异替代RL变异
                child1 = self.rl_mutate(child1)
                child2 = self.rl_mutate(child2)
                new_population.extend([child1, child2])
            self.population = new_population[: self.population_size]  # 更新种群
            # 评估当前种群最佳解
            current_best = min(self.population, key=lambda ind: self.evaluate_fitness(ind))
            current_best_fitness = self.evaluate_fitness(current_best)
            if current_best_fitness < best_fitness:
                best_fitness = current_best_fitness
                best_solution = copy.deepcopy(current_best.fbs_model)
                best_dqn_model = self.dqn_model
                best_generation = generation
                fast_time = datetime.datetime.now()
                best_time = (fast_time - start_time).total_seconds()  # 记录从开始到获得最佳解的时间(秒)
                if self.is_train:
                    # 每当找到更好的解时，保存DQN模型
                    model_path = f"models/{self.env.instance}_dqn_model_gen_{generation}_fitness_{best_fitness:.2f}_{self.train_id}"
                    logger.info(f"保存DQN模型到: {model_path}")
                    self.dqn_model.save(model_path)
                else:
                    logger.info(f"第{generation}代，最佳适应度: {best_fitness}")
                
            if generation % 10 == 0:
                logger.info(f"Generation {generation}, Best Fitness: {best_fitness}")

        end_time = datetime.datetime.now()
        if self.is_train:   
            # 最终保存DQN模型
            final_model_path = f"models/{self.env.instance}_dqn_model_final_{self.train_id}"
            logger.info(f"保存最终DQN模型到: {final_model_path}")
            self.dqn_model.save(final_model_path)
        
        # 记录训练相关信息
        logger.info(f"最佳适应度: {best_fitness}, 在第 {best_generation} 代获得")
        logger.info(f"总训练时间: {end_time - start_time}")
        logger.info(f"获得最佳解用时: {best_time} 秒")
        
        return best_solution, best_fitness, start_time, end_time, fast_time, best_time

In [12]:
if __name__ == "__main__":
    # 检查可用的设备
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    logger.info(f"使用设备: {device}")
    # 实验参数
    exp_instance = "Du62"
    exp_algorithm = "RL+遗传算法"
    exp_remark = "K分初始解v3-奖励函数v1-测试"
    exp_number = 20  # 运行次数
    is_exp = True  # 是否进行实验
    is_train = False  # 是否进行训练
    model_path = "/Users/maoyan/Codes/Python/gym-flp-fbs/models/Du62_dqn_model_final"  # 模型路径
    # 算法参数
    population_size = 50
    crossover_rate = 0.8
    mutation_rate = 0.1
    max_generations = 62 * 62  # 最大迭代次数
    dqn_train_freq = 4  # DQN训练频率
    dqn_learning_starts = 1000  # DQN开始学习的步数
    
    # 创建环境和GA实例
    env = gym.make("FbsEnv-v0", instance=exp_instance)
    ga = GeneticAlgorithm(
        env=env,
        population_size=population_size,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        max_generations=max_generations,
        dqn_train_freq=dqn_train_freq,
        dqn_learning_starts=dqn_learning_starts,
        device=device,
        is_train=is_train,
        model_path=model_path
    )
    if is_exp:
        for i in range(exp_number):
            logger.info(f"第{i+1}次实验")
            env = gym.make("FbsEnv-v0", instance=exp_instance)
            ga = GeneticAlgorithm(
                env=env,
                population_size=population_size,
                crossover_rate=crossover_rate,
                mutation_rate=mutation_rate,
                max_generations=max_generations,
                dqn_train_freq=dqn_train_freq,
                dqn_learning_starts=dqn_learning_starts,
                device=device,
                is_train=is_train,
                model_path=model_path
            )
            best_solution, best_fitness, exp_start_time, exp_end_time, exp_fast_time, best_time = ga.run()
            print(f"Best Solution: {best_solution.array_2d}, Best Fitness: {best_fitness}")
            ExperimentsUtil.save_experiment_result(
                exp_instance=exp_instance,
                exp_algorithm=exp_algorithm,
                exp_iterations=max_generations,
                exp_solution=best_solution.array_2d,
                exp_fitness=best_fitness,
                exp_start_time=exp_start_time,
                exp_fast_time=exp_fast_time,
                exp_end_time=exp_end_time,
                exp_remark=exp_remark,
                exp_best_time=best_time  # 添加最佳解的训练时间
            )
    else:
        best_solution, best_fitness, exp_start_time, exp_end_time, exp_fast_time, best_time = ga.run()
        logger.info(f"Best Solution: {best_solution.array_2d}, Best Fitness: {best_fitness}")
        logger.info(f"获得最佳解用时: {best_time} 秒")
        env.reset(fbs_model=best_solution)
        env.render()

2025-05-14 21:21:23.100 | INFO     | __main__:<module>:9 - 使用设备: mps
/Users/maoyan/miniconda3/envs/gym-fbs/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
2025-05-14 21:21:27.248 | INFO     | __main__:<module>:42 - 第1次实验
/Users/maoyan/miniconda3/envs/gym-fbs/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
2025-05-14 21:21:31.061 | INFO     | __main__:run:200 - 第0代，最佳适应度: 4671197.051847599
2025-05-14 21:21:31.061 | IN

Best Solution: [[13, 33, 39, 34, 7, 48, 15], [41, 22, 50, 47, 58, 46], [54, 49, 52, 11, 29, 9], [4, 5, 40, 16, 27, 55, 31], [23, 30, 38, 2, 43, 21, 57, 59, 37], [19, 1, 18, 28, 56, 61, 12, 24, 45], [10, 36, 20, 32, 60, 44, 8, 26], [6, 62, 51, 42, 25, 3, 35, 53, 14, 17]], Best Fitness: 3745652.909576573


2025-05-14 21:29:56.246 | INFO     | __main__:run:200 - 第0代，最佳适应度: 4501364.102171825
2025-05-14 21:29:56.246 | INFO     | __main__:run:203 - Generation 0, Best Fitness: 4501364.102171825
2025-05-14 21:29:56.361 | INFO     | __main__:run:200 - 第1代，最佳适应度: 4464924.273786854
2025-05-14 21:29:56.550 | INFO     | __main__:run:200 - 第2代，最佳适应度: 4454985.166427017
2025-05-14 21:29:56.662 | INFO     | __main__:run:200 - 第3代，最佳适应度: 4365438.180720659
2025-05-14 21:29:56.767 | INFO     | __main__:run:200 - 第4代，最佳适应度: 4340864.920512501
2025-05-14 21:29:56.856 | INFO     | __main__:run:200 - 第5代，最佳适应度: 4336071.607067269
2025-05-14 21:29:57.040 | INFO     | __main__:run:200 - 第6代，最佳适应度: 4327966.089795114
2025-05-14 21:29:57.146 | INFO     | __main__:run:200 - 第7代，最佳适应度: 4303756.346183245
2025-05-14 21:29:57.252 | INFO     | __main__:run:200 - 第8代，最佳适应度: 4300105.807899818
2025-05-14 21:29:57.442 | INFO     | __main__:run:200 - 第9代，最佳适应度: 4291293.26164601
2025-05-14 21:29:57.565 | INFO     | __main__:run

Best Solution: [[6, 48, 62, 53, 54, 31, 15], [13, 35, 2, 59, 40, 46], [42, 1, 44, 27, 58], [19, 36, 61, 32, 43, 29, 34], [10, 24, 56, 28, 38, 52, 5, 37], [23, 18, 60, 21, 16, 47, 8], [4, 30, 3, 20, 11, 12, 33], [41, 22, 25, 50, 57, 51, 55], [17, 45, 39, 49, 26, 7, 9, 14]], Best Fitness: 3705008.022989612


2025-05-14 21:38:29.984 | INFO     | __main__:run:200 - 第0代，最佳适应度: 4628735.738983258
2025-05-14 21:38:29.985 | INFO     | __main__:run:203 - Generation 0, Best Fitness: 4628735.738983258
2025-05-14 21:38:30.082 | INFO     | __main__:run:200 - 第1代，最佳适应度: 4612227.161674997
2025-05-14 21:38:30.207 | INFO     | __main__:run:200 - 第2代，最佳适应度: 4542361.689793406
2025-05-14 21:38:30.522 | INFO     | __main__:run:200 - 第4代，最佳适应度: 4519330.139590046
2025-05-14 21:38:30.635 | INFO     | __main__:run:200 - 第5代，最佳适应度: 4497128.72475996
2025-05-14 21:38:30.816 | INFO     | __main__:run:200 - 第6代，最佳适应度: 4481545.589992452
2025-05-14 21:38:31.197 | INFO     | __main__:run:200 - 第9代，最佳适应度: 4471694.4704110455
2025-05-14 21:38:31.287 | INFO     | __main__:run:203 - Generation 10, Best Fitness: 4471694.4704110455
2025-05-14 21:38:31.371 | INFO     | __main__:run:200 - 第11代，最佳适应度: 4461691.480656437
2025-05-14 21:38:31.629 | INFO     | __main__:run:200 - 第13代，最佳适应度: 4452541.873091389
2025-05-14 21:38:31.845 | I

...

Best Solution: [[6, 13, 42, 48, 31, 7, 15], [62, 36, 35, 52, 44, 46], [53, 1, 2, 27, 47, 34], [19, 58, 61, 56, 59, 29, 54], [37, 32, 60, 28, 40, 16, 43, 5, 14], [8, 30, 18, 24, 21, 57, 11, 50, 33], [41, 4, 3, 20, 38, 25, 49, 39], [17, 10, 23, 22, 55, 12, 51, 26, 45, 9]], Best Fitness: 3733760.152549389


2025-05-15 07:56:06.457 | INFO     | __main__:run:200 - 第0代，最佳适应度: 4521350.972464231
2025-05-15 07:56:06.457 | INFO     | __main__:run:203 - Generation 0, Best Fitness: 4521350.972464231
2025-05-15 07:56:06.687 | INFO     | __main__:run:200 - 第2代，最佳适应度: 4518957.098369812
2025-05-15 07:56:06.856 | INFO     | __main__:run:200 - 第3代，最佳适应度: 4480893.386257287
2025-05-15 07:56:07.048 | INFO     | __main__:run:200 - 第5代，最佳适应度: 4450488.876633468
2025-05-15 07:56:07.223 | INFO     | __main__:run:200 - 第6代，最佳适应度: 4439060.446090665
2025-05-15 07:56:07.440 | INFO     | __main__:run:200 - 第8代，最佳适应度: 4401561.658887226
2025-05-15 07:56:07.606 | INFO     | __main__:run:200 - 第9代，最佳适应度: 4401187.265073026
2025-05-15 07:56:07.699 | INFO     | __main__:run:203 - Generation 10, Best Fitness: 4401187.265073026
2025-05-15 07:56:07.982 | INFO     | __main__:run:200 - 第12代，最佳适应度: 4393252.945370279
2025-05-15 07:56:08.074 | INFO     | __main__:run:200 - 第13代，最佳适应度: 4387027.654925018
2025-05-15 07:56:08.198 | IN

Best Solution: [[19, 14, 54, 40, 62, 33, 17], [10, 30, 32, 38, 25, 9], [23, 3, 2, 34, 49], [41, 1, 61, 27, 43, 52, 39], [36, 18, 28, 56, 57, 50, 11, 31], [13, 24, 59, 12, 16, 29, 37], [4, 55, 20, 21, 5, 47, 7], [22, 42, 60, 58, 44, 51, 48], [6, 45, 53, 35, 8, 26, 46, 15]], Best Fitness: 3739481.4929034906


2025-05-15 08:04:34.861 | INFO     | __main__:run:200 - 第0代，最佳适应度: 4677685.612752452
2025-05-15 08:04:34.861 | INFO     | __main__:run:203 - Generation 0, Best Fitness: 4677685.612752452
2025-05-15 08:04:35.025 | INFO     | __main__:run:200 - 第1代，最佳适应度: 4563219.875706512
2025-05-15 08:04:35.414 | INFO     | __main__:run:200 - 第4代，最佳适应度: 4534382.100822406
2025-05-15 08:04:35.523 | INFO     | __main__:run:200 - 第5代，最佳适应度: 4518621.734123123
2025-05-15 08:04:35.640 | INFO     | __main__:run:200 - 第6代，最佳适应度: 4516348.171456085
2025-05-15 08:04:35.836 | INFO     | __main__:run:200 - 第7代，最佳适应度: 4515861.742710259
2025-05-15 08:04:35.942 | INFO     | __main__:run:200 - 第8代，最佳适应度: 4493950.429524393
2025-05-15 08:04:36.051 | INFO     | __main__:run:200 - 第9代，最佳适应度: 4476316.142049958
2025-05-15 08:04:36.225 | INFO     | __main__:run:200 - 第10代，最佳适应度: 4472389.537813992
2025-05-15 08:04:36.225 | INFO     | __main__:run:203 - Generation 10, Best Fitness: 4472389.537813992
2025-05-15 08:04:36.342 | INF

Best Solution: [[15, 62, 42, 31, 19, 13, 6], [48, 10, 57, 40, 36, 25, 7], [54, 58, 27, 38, 28, 2, 32, 56, 12, 20, 49, 22], [4, 61, 52, 11, 16, 30, 50, 47, 39], [53, 51, 1, 43, 59, 21, 5, 44, 33], [46, 41, 8, 3, 29, 60, 35, 45], [17, 14, 37, 23, 24, 18, 55, 34, 26, 9]], Best Fitness: 3760545.9964139815
